In [1]:
import pandas as pd

# Cleaning Stage

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
# Remove the space for 'FALSE POSITIVE'
new = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[new, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1            CONFIRMED
2       False_Positive
3       False_Positive
4            CONFIRMED
             ...      
9559    False_Positive
9560    False_Positive
9561         CANDIDATE
9562    False_Positive
9563    False_Positive
Name: koi_disposition, Length: 8744, dtype: object

# Split Train and Test Data Sets

In [4]:
# Asssign the X and y values
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(8744, 40) (8744,)


In [5]:
# Split dataframe training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3206,0,1,1,0,31.805143,0.000042,-0.000042,186.38905,0.00105,-0.00105,...,-161.0,4.545,0.044,-0.176,0.863,0.217,-0.072,298.04453,40.086361,14.517
3954,0,0,0,0,24.560711,0.000375,-0.000375,146.79380,0.01610,-0.01610,...,-135.0,4.192,0.137,-0.125,1.499,0.272,-0.245,289.92145,46.744560,12.805
1410,0,0,0,0,7.560522,0.000026,-0.000026,134.47889,0.00270,-0.00270,...,-211.0,4.503,0.052,-0.208,0.940,0.282,-0.094,283.84515,44.609089,15.986
5865,0,0,0,0,4.644901,0.000038,-0.000038,133.67436,0.00786,-0.00786,...,-85.0,4.540,0.052,-0.017,0.770,0.027,-0.046,282.34305,48.340778,14.480
340,0,0,1,1,2.037441,0.000010,-0.000010,133.59962,0.00390,-0.00390,...,-207.0,4.415,0.087,-0.203,1.015,0.312,-0.134,295.79526,47.663960,14.187


# Normalization Stage

Scale the data using the MinMaxScaler

In [7]:
# Normalization process through MinMaxScaler funtion
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to categorical
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Deep Learning Model Stage

In [10]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model = Sequential()
# Add two layers of nodes
model.add(Dense(units=120, activation='relu', input_dim=40))
model.add(Dense(units=120, activation='relu'))
# Add the output layer
model.add(Dense(units=3, activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               4920      
_________________________________________________________________
dense_1 (Dense)              (None, 120)               14520     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 363       
Total params: 19,803
Trainable params: 19,803
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Fit the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
205/205 - 1s - loss: 0.4855 - accuracy: 0.7704
Epoch 2/100
205/205 - 0s - loss: 0.3615 - accuracy: 0.8192
Epoch 3/100
205/205 - 0s - loss: 0.3533 - accuracy: 0.8214
Epoch 4/100
205/205 - 0s - loss: 0.3403 - accuracy: 0.8352
Epoch 5/100
205/205 - 0s - loss: 0.3305 - accuracy: 0.8445
Epoch 6/100
205/205 - 0s - loss: 0.3318 - accuracy: 0.8391
Epoch 7/100
205/205 - 0s - loss: 0.3238 - accuracy: 0.8484
Epoch 8/100
205/205 - 0s - loss: 0.3166 - accuracy: 0.8574
Epoch 9/100
205/205 - 0s - loss: 0.3097 - accuracy: 0.8605
Epoch 10/100
205/205 - 0s - loss: 0.3075 - accuracy: 0.8606
Epoch 11/100
205/205 - 0s - loss: 0.3089 - accuracy: 0.8600
Epoch 12/100
205/205 - 0s - loss: 0.2962 - accuracy: 0.8686
Epoch 13/100
205/205 - 0s - loss: 0.2925 - accuracy: 0.8718
Epoch 14/100
205/205 - 0s - loss: 0.2847 - accuracy: 0.8748
Epoch 15/100
205/205 - 0s - loss: 0.2934 - accuracy: 0.8679
Epoch 16/100
205/205 - 0s - loss: 0.2907 - accuracy: 0.8684
Epoch 17/100
205/205 - 0s - loss: 0.2850 - accura

# Model Evaluation Stage

In [14]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

69/69 - 0s - loss: 0.2559 - accuracy: 0.8866
Normal Neural Network - Loss: 0.2559107542037964, Accuracy: 0.886550784111023


# Predictions Stage

In [15]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

d:\bootcamp\bootcamp\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [16]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'False_Positive' 'False_Positive'
 'CONFIRMED']
Actual Labels: ['False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED']
